In [1]:
!pip install pyannote.audio

In [2]:
!pip install torchvision

In [3]:
! pip install python-dotenv
! pip install pydub
! pip install pandas
! pip install psycopg2
! pip install google-auth google-auth-httplib2 google-api-python-client google-auth-oauthlib

In [4]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import split_audio_files

Lightning automatically upgraded your loaded checkpoint from v1.1.3 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/059e96f964841d40f1a5e755bb7223f76666bba4/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.4.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [5]:
import requests
import os
import subprocess

def download_url_file(url, local_filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_filename.strip(), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded {local_filename.strip()}")
    else:
        print(f"Failed to download {local_filename.strip()} from {url}. Status code: {response.status_code}")


In [6]:
# Ensure directories exist
os.makedirs('audio_from_url', exist_ok=True)

# Example usage:
from_id = 163 
to_id = 201

df = read_spreadsheet(sheet_id="1SjlnWsr6YVhBIW6EaS0ITmboDmEuj2VnxGRNL50OPx4")

for index, row in df.iterrows():
    if not isinstance(row['ID'], str) or not isinstance(row['YouTube link'], str):
        break
    id = row['ID']
    url_path = row['YouTube link']
    audio_filename = f"audio_from_url/{id}.mp3".strip()
    sr_no = row.iloc[0]
    if sr_no >= from_id and sr_no <= to_id:
        if os.path.exists(audio_filename):
            print(f"Audio file {audio_filename} already exists. Skipping extraction.")
            continue
        if not os.path.exists(audio_filename):
            download_url_file(url_path, audio_filename)


Downloaded audio_from_url/STT_HS0163.mp3
Downloaded audio_from_url/STT_HS0164.mp3
Downloaded audio_from_url/STT_HS0165.mp3
Downloaded audio_from_url/STT_HS0166.mp3
Downloaded audio_from_url/STT_HS0167.mp3
Downloaded audio_from_url/STT_HS0168.mp3
Downloaded audio_from_url/STT_HS0169.mp3
Downloaded audio_from_url/STT_HS0170.mp3
Downloaded audio_from_url/STT_HS0171.mp3
Downloaded audio_from_url/STT_HS0172.mp3
Downloaded audio_from_url/STT_HS0173.mp3
Downloaded audio_from_url/STT_HS0174.mp3
Downloaded audio_from_url/STT_HS0175.mp3
Downloaded audio_from_url/STT_HS0176.mp3
Downloaded audio_from_url/STT_HS0177.mp3
Downloaded audio_from_url/STT_HS0178.mp3
Downloaded audio_from_url/STT_HS0179.mp3
Downloaded audio_from_url/STT_HS0180.mp3
Downloaded audio_from_url/STT_HS0181.mp3
Downloaded audio_from_url/STT_HS0182.mp3
Downloaded audio_from_url/STT_HS0183.mp3
Downloaded audio_from_url/STT_HS0184.mp3
Downloaded audio_from_url/STT_HS0185.mp3
Downloaded audio_from_url/STT_HS0186.mp3
Downloaded audio

In [7]:
import subprocess
import os

def convert_to_16K(input_file, output_file):
    # Check if the output file already exists
    if os.path.exists(output_file):
        print(f"Output file {output_file} already exists. Skipping conversion.")
        return False
    
    # Construct the ffmpeg command
    ffmpeg_command = [
        "ffmpeg",
        "-i", input_file,
        "-f", "wav",            # Force input format as WAV
        "-bitexact",            # Preserve exact precision
        "-acodec", "pcm_s16le", # Audio codec: PCM signed 16-bit little-endian
        "-ac", "1",             # Mono channel
        "-ar", "16000",         # 16 kHz sample rate
        output_file,
        "-y"                    # Overwrite output file if it exists
    ]
    
    # Join the ffmpeg command into a single string for subprocess
    ffmpeg_command_str = " ".join(ffmpeg_command)
    
    try:
        # Run ffmpeg command using subprocess
        subprocess.run(ffmpeg_command_str, shell=True, check=True, capture_output=True)
        print(f"Conversion successful: {input_file} -> {output_file}")
        return True
    except subprocess.CalledProcessError as e:
        print(f"Conversion failed: {e}")
        return False


In [8]:
from pydub import AudioSegment
import os

def convert_mp3_to_wav(mp3_file, output_folder):
    # Ensure output folder exists, create if necessary
    os.makedirs(output_folder, exist_ok=True)
    
    # Load MP3 file
    audio = AudioSegment.from_mp3(mp3_file)
    
    # Define output file path (replace .mp3 with .wav)
    wav_file = os.path.join(output_folder, os.path.splitext(os.path.basename(mp3_file))[0] + ".wav")
    
    # Export audio to WAV format
    audio.export(wav_file, format="wav")
    
    print(f"Converted {mp3_file} to {wav_file}")

def batch_convert_mp3_to_wav(input_dir, output_dir):
    # Ensure output directory exists, create if necessary
    os.makedirs(output_dir, exist_ok=True)
    
    # List all files in the input directory
    files = os.listdir(input_dir)
    
    # Filter only .mp3 files
    mp3_files = [f for f in files if f.startswith("STT_HS") and f.endswith(".mp3")]
    
    print(mp3_files)
    # Iterate over each MP3 file and convert to WAV
    for mp3_file in mp3_files:
        mp3_path = os.path.join(input_dir, mp3_file)
        convert_mp3_to_wav(mp3_path, output_dir)


In [9]:
# Example usage:
input_directory = "audio_from_url"  # Replace with your input directory path
output_directory = "full_audio_wav"  # Replace with your desired output directory path

batch_convert_mp3_to_wav(input_directory, output_directory)


['STT_HS0182.mp3', 'STT_HS0169.mp3', 'STT_HS0179.mp3', 'STT_HS0191.mp3', 'STT_HS0163.mp3', 'STT_HS0183.mp3', 'STT_HS0167.mp3', 'STT_HS0178.mp3', 'STT_HS0201.mp3', 'STT_HS0168.mp3', 'STT_HS0186.mp3', 'STT_HS0177.mp3', 'STT_HS0189.mp3', 'STT_HS0196.mp3', 'STT_HS0187.mp3', 'STT_HS0199.mp3', 'STT_HS0190.mp3', 'STT_HS0181.mp3', 'STT_HS0194.mp3', 'STT_HS0198.mp3', 'STT_HS0175.mp3', 'STT_HS0165.mp3', 'STT_HS0164.mp3', 'STT_HS0185.mp3', 'STT_HS0192.mp3', 'STT_HS0176.mp3', 'STT_HS0166.mp3', 'STT_HS0173.mp3', 'STT_HS0193.mp3', 'STT_HS0172.mp3', 'STT_HS0188.mp3', 'STT_HS0174.mp3', 'STT_HS0200.mp3', 'STT_HS0170.mp3', 'STT_HS0171.mp3', 'STT_HS0195.mp3', 'STT_HS0180.mp3', 'STT_HS0184.mp3', 'STT_HS0197.mp3']
Converted audio_from_url/STT_HS0182.mp3 to full_audio_wav/STT_HS0182.wav
Converted audio_from_url/STT_HS0169.mp3 to full_audio_wav/STT_HS0169.wav
Converted audio_from_url/STT_HS0179.mp3 to full_audio_wav/STT_HS0179.wav
Converted audio_from_url/STT_HS0191.mp3 to full_audio_wav/STT_HS0191.wav
Conve

In [10]:
from pathlib import Path
path = Path('./full_audio_wav/')
full_audios = set([e.stem for e in path.iterdir() if e.is_file()])

In [11]:
print(full_audios)

{'STT_HS0171', 'STT_HS0190', 'STT_HS0194', 'STT_HS0167', 'STT_HS0168', 'STT_HS0164', 'STT_HS0198', 'STT_HS0188', 'STT_HS0195', 'STT_HS0183', 'STT_HS0177', 'STT_HS0182', 'STT_HS0173', 'STT_HS0187', 'STT_HS0175', 'STT_HS0180', 'STT_HS0166', 'STT_HS0197', 'STT_HS0186', 'STT_HS0176', 'STT_HS0191', 'STT_HS0199', 'STT_HS0174', 'STT_HS0181', 'STT_HS0170', 'STT_HS0196', 'STT_HS0184', 'STT_HS0178', 'STT_HS0201', 'STT_HS0193', 'STT_HS0165', 'STT_HS0189', 'STT_HS0169', 'STT_HS0200', 'STT_HS0185', 'STT_HS0179', 'STT_HS0163', 'STT_HS0192', 'STT_HS0172'}


In [12]:
os.makedirs('audio_16k', exist_ok=True)
for full_audio in full_audios:
    print(full_audio)
    input_file = f"full_audio_wav/{full_audio}.wav"
    output_file = f"audio_16k/{full_audio}.wav"
    convert_to_16K(input_file, output_file)

STT_HS0171
Conversion successful: full_audio_wav/STT_HS0171.wav -> audio_16k/STT_HS0171.wav
STT_HS0190
Conversion successful: full_audio_wav/STT_HS0190.wav -> audio_16k/STT_HS0190.wav
STT_HS0194
Conversion successful: full_audio_wav/STT_HS0194.wav -> audio_16k/STT_HS0194.wav
STT_HS0167
Conversion successful: full_audio_wav/STT_HS0167.wav -> audio_16k/STT_HS0167.wav
STT_HS0168
Conversion successful: full_audio_wav/STT_HS0168.wav -> audio_16k/STT_HS0168.wav
STT_HS0164
Conversion successful: full_audio_wav/STT_HS0164.wav -> audio_16k/STT_HS0164.wav
STT_HS0198
Conversion successful: full_audio_wav/STT_HS0198.wav -> audio_16k/STT_HS0198.wav
STT_HS0188
Conversion successful: full_audio_wav/STT_HS0188.wav -> audio_16k/STT_HS0188.wav
STT_HS0195
Conversion successful: full_audio_wav/STT_HS0195.wav -> audio_16k/STT_HS0195.wav
STT_HS0183
Conversion successful: full_audio_wav/STT_HS0183.wav -> audio_16k/STT_HS0183.wav
STT_HS0177
Conversion successful: full_audio_wav/STT_HS0177.wav -> audio_16k/STT

In [13]:
import shutil
import os
for file in os.listdir("audio_16k"):
    if not os.path.exists( os.path.join("./full_audio/",file)):
        os.makedirs("./full_audio/", exist_ok=True)
        print("coppying ", file)
        shutil.copy(os.path.join("audio_16k",file), os.path.join("./full_audio/",file))

coppying  STT_HS0201.wav
coppying  STT_HS0173.wav
coppying  STT_HS0187.wav
coppying  STT_HS0197.wav
coppying  STT_HS0191.wav
coppying  STT_HS0166.wav
coppying  STT_HS0163.wav
coppying  STT_HS0175.wav
coppying  STT_HS0185.wav
coppying  STT_HS0189.wav
coppying  STT_HS0194.wav
coppying  STT_HS0172.wav
coppying  STT_HS0182.wav
coppying  STT_HS0198.wav
coppying  STT_HS0168.wav
coppying  STT_HS0174.wav
coppying  STT_HS0165.wav
coppying  STT_HS0196.wav
coppying  STT_HS0188.wav
coppying  STT_HS0180.wav
coppying  STT_HS0164.wav
coppying  STT_HS0179.wav
coppying  STT_HS0170.wav
coppying  STT_HS0183.wav
coppying  STT_HS0176.wav
coppying  STT_HS0195.wav
coppying  STT_HS0184.wav
coppying  STT_HS0193.wav
coppying  STT_HS0169.wav
coppying  STT_HS0190.wav
coppying  STT_HS0186.wav
coppying  STT_HS0181.wav
coppying  STT_HS0192.wav
coppying  STT_HS0171.wav
coppying  STT_HS0177.wav
coppying  STT_HS0178.wav
coppying  STT_HS0200.wav
coppying  STT_HS0199.wav
coppying  STT_HS0167.wav


In [14]:
split_audio_files('STT_HS', 'wav')

['STT_HS0201.wav', 'STT_HS0173.wav', 'STT_HS0187.wav', 'STT_HS0197.wav', 'STT_HS0191.wav', 'STT_HS0166.wav', 'STT_HS0163.wav', 'STT_HS0175.wav', 'STT_HS0185.wav', 'STT_HS0189.wav', 'STT_HS0194.wav', 'STT_HS0172.wav', 'STT_HS0182.wav', 'STT_HS0198.wav', 'STT_HS0168.wav', 'STT_HS0174.wav', 'STT_HS0165.wav', 'STT_HS0196.wav', 'STT_HS0188.wav', 'STT_HS0180.wav', 'STT_HS0164.wav', 'STT_HS0179.wav', 'STT_HS0170.wav', 'STT_HS0183.wav', 'STT_HS0176.wav', 'STT_HS0195.wav', 'STT_HS0184.wav', 'STT_HS0193.wav', 'STT_HS0169.wav', 'STT_HS0190.wav', 'STT_HS0186.wav', 'STT_HS0181.wav', 'STT_HS0192.wav', 'STT_HS0171.wav', 'STT_HS0177.wav', 'STT_HS0178.wav', 'STT_HS0200.wav', 'STT_HS0199.wav', 'STT_HS0167.wav']


  0%|          | 0/39 [00:00<?, ?it/s]

STT_HS0201.wav
./full_audio/STT_HS0201.wav STT_HS0201
1 3.36 5042.84 8400.97 vad
2 8.69 11894.09 20584.72 vad
3 3.14 23090.72 26226.72 split
4 2.88 29490.72 32370.72 split
5 2.72 33906.72 36626.72 split
6 3.30 38228.84 41524.84 split
7 2.02 45844.84 47860.84 split
8 3.97 48276.84 52244.84 split
9 2.27 56180.84 58452.84 split
10 2.78 62068.84 64852.84 split
11 6.04 77537.84 83579.09 vad
12 7.24 85637.84 92877.22 vad
13 8.72 93417.22 102141.59 vad
14 3.80 102512.84 106309.72 vad
15 2.03 116130.97 118155.97 vad
16 4.52 119354.09 123876.59 vad
17 2.82 124939.72 127755.72 split
18 4.10 128043.72 132139.72 split
19 2.24 132363.72 134603.72 split
20 2.50 138912.22 141408.22 split
21 6.02 156647.84 162672.22 vad
22 5.01 165456.59 170468.47 vad
23 5.64 173759.09 179395.34 vad
24 3.26 182179.72 185436.59 vad
25 2.90 185875.34 188777.84 vad
26 2.04 189300.97 191342.84 vad
27 2.24 195765.84 198005.84 split
28 2.43 198421.84 200853.84 split
29 3.62 201557.84 205173.84 split
30 4.72 208487.84 213212

KeyboardInterrupt: 

In [ ]:
collect_segments('STT_HS', 'after_split', 'segments_hs')

In [ ]:
! aws s3 cp segments_hs  s3://monlam.ai.stt/wav16k/ --recursive